# Discussion physique

Dans cette première partie, nous nous concentrerons sur la propagation d'une fake news par le seul biais d'une interaction entre agents physiquement proches. Cette discussion pourra avoir lieu **entre 2 à 9 personnes**.

On rappelle la description d'un agent : 

|Caractéristique||Nom    |Type   |Intervalle     |Valeur initiale  | Fixe ?    |
|:-------------:||:-----:|:-----:|:-------------:|:---------------:|:---------:|
|**Nom** : nom de l'agent (clé du dict)                                  ||**nom**    |str    |[str(0),str(N)]|str(k)|Oui|
|**Opinion** : y croit (1) ou non (0)                                    ||**op**     |int    |[0,1]          |0     |Non|
|**Déjà vu** : a déjà été confronté à la news (True) ou pas (False)      ||**deja_vu**|bool   |[True,False]   |False |Non|
|**Force de persuasion** : capacité à convaincre qqun (accrue qd tend vers 1) ||**fp**|float  |[0,1]          |Random|Oui|
|**Conviction** : caractérise la conviction à la news (tend vers 1) ou non (tend vers 0)||**p**|float  |[0,1] |0.5   |Non| 

**Processus de discussion détaillé :** 
- **1ère étape : Mise à jour des convictions**  

A partir d'une matrice représentant l'ensemble de la population, nous sélectionnerons un agent et son voisinage pour créer une liste de 2 à 9 personnes : la **liste discussion**.
- si il n'y a que _2 personnes_ dans la liste discussion, seule la **force de persuasion** influera et ce sur un seul agent, celui dont la force de persuasion est la plus faible. Celle-ci est régie par la loi suivante : lorsque $  fp_i $ < $  fp_j $ on applique la formule: $ p_i \leftarrow p_i + fp_i  \cdot ( p_j - p_i ) $
- si la discussion comprend _de 3 à 9 personnes_, ce sera le **voisinage** qui influera sur la conviction de chacun selon la loi suivante. Chaque individu modifie sa conviction en fonction de la moyenne des convictions des autres pondérée de leur force de persuasion ($ moy = \frac{\sum p_i*fp_i}{\sum fp_i} $) et du seuil de persuasion $\sigma$ : si $ |p_i-moy| < \sigma $, alors $ p_i \leftarrow moy $ sinon, $p_i$ ne change pas.  

De plus, la conviction sera affectée par la **crédibilité de la news** $news$ par la loi suivante : $ p_i \leftarrow p_i + fp_i \cdot ( news - p_i ) $ 
- **2ème étape : Mise à jour des opinions**  
Pour chaque agent, on modifie son opinion en fonction de sa conviction : si $ p_i  > 0.5 $ , $op_i$ passe de $0$ à $1$ et inversement.

# Initialisation du système

Nous initialisons **une matrice** de $ N\_1dim*N\_1dim=N $ individus (nous nous aviserons donc de choisir N tel qu'il soit un carré parfait). De plus, nous modélisons un réseau discussion par **un dictionaire**, qui associe à chaque individu un tuple, de la forme $ dict[str:tuple[int,bool,float,float]] $. Celui-ci représentera intégralement un agent et ses caractéristiques.

In [1]:
import numpy as np
import random

#description d'un agent : nom:(opinion, deja_vu , force_de_persuasion , conviction)
#                         nom:(   op  , deja_vu ,       fp            ,     p     )
N_1dim = 11    #nombre de membres du réseau sur une dimension (racine de N)
N = N_1dim**2  #nombre de membres du réseau discussion physique

#Initialisation :
Reseau_discu={str(k):(0,False,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Noms_reseau_discu=[k for k in range(N)]

#Réseau à valeurs quelconques (au milieu de la simulation par exemple)
Test_reseau={str(k):(np.random.choice([0,1]),np.random.choice([False,True]),np.random.random_sample(),np.random.random_sample()) for k in range(1,N+1)}
print('Réseau aléatoire :\n',Test_reseau,'\n')

#Représentation des agents en 2 dimensions (sur une matrice) :
Agents=np.arange(1,N_1dim*N_1dim+1).reshape(N_1dim,N_1dim)
print('Agents :\n',Agents)

Réseau aléatoire :
 {'1': (0, False, 0.7346610644792022, 0.6477871249978855), '2': (0, False, 0.45525028831027037, 0.26946477240128097), '3': (1, False, 0.912963782459134, 0.22072990873325526), '4': (1, False, 0.8862085155815076, 0.757202584897424), '5': (1, False, 0.9825138050903007, 0.9631282079132993), '6': (1, False, 0.028747756157672333, 0.900310729498335), '7': (0, True, 0.002107295007200416, 0.38632638413879294), '8': (0, False, 0.19336685203411685, 0.41955404803995444), '9': (1, True, 0.19662693644421148, 0.5973916975025212), '10': (1, True, 0.8032316100869032, 0.13855506366579617), '11': (0, False, 0.4008885277405718, 0.419208628445233), '12': (0, True, 0.23314933782228642, 0.27837861409227727), '13': (1, False, 0.689953603962745, 0.9893369133950484), '14': (0, False, 0.7325228084721132, 0.9629208077202279), '15': (0, True, 0.13208159666548458, 0.9773101286645968), '16': (0, True, 0.9943512396814158, 0.3566780732467155), '17': (1, False, 0.04211193981001771, 0.875672204483548)

# Facteurs modifiant les paramètres

## Influence de la crédibilité de la news sur p

In [35]:
#Influence crébibilité news sur le p d'une personne
news = np.random.random_sample()
print('Crédibilité de l\'info:',news)

def influ_credib_news(news,p,fp):
    #news: crédibilité de la news
    #p: param déterminant le chgt ou non de l'opinion    
    i = p
    i = i+fp*(news-i)
    return i

print(influ_credib_news(news,0.5,0.75))

Crédibilité de l'info: 0.5840746791567739
0.5630560093675805


## Influence du voisinage sur p

In [31]:
#Sélection d'un agent et de son voisinage pour établir un discussion
def selection_voisinage_bis(Matrice,n_1dim):
    '''Sélection aléatoire des voisinages d'un individu
    '''
    L=[]                               #liste voisinage (pers susceptibles de discuter)
    Ldiscu=[]                          #liste discussion (pers qui vont discuter)
    size=np.random.choice(range(2,10)) #nb pers discussion
    
    ic=np.random.choice(range(n_1dim)) #ic,jc : indices de l'agent central
    jc=np.random.choice(range(n_1dim))
    a_central=Matrice[ic,jc]
    print('Agent central:',a_central)
    
    if ic>0 and ic<(n_1dim-1) and jc>0 and jc<(n_1dim-1):
        L=np.reshape(Matrice[ic-1 : ic+2 , jc-1 : jc+2] , 9)        
    elif ic==0 and jc>0 and jc<(n_1dim-1):          #a_central sur arête sup
        L=[(Matrice[ic,jc-1 : jc+2])
           ,(Matrice[ic+1,jc-1 : jc+2])
           ,(Matrice[n_1dim-1, jc-1 : jc+2])]
        L=np.reshape(L,9)
    elif ic==(n_1dim-1) and jc>0 and jc<(n_1dim-1): #a_central sur arête inf
        L=[(Matrice[ic,jc-1 : jc+2])
           ,(Matrice[ic-1,jc-1 : jc+2])
           ,(Matrice[0, jc-1 : jc+2])]
        L=np.reshape(L,9)        
    elif jc==0 and ic>0 and ic<(n_1dim-1):          #a_central sur arête gauche
        L=[(Matrice[ic-1 : ic+2,jc])
           ,(Matrice[ic-1 : ic+2,jc+1])
           ,(Matrice[ic-1 : ic+2,n_1dim-1])]
        L=np.reshape(L,9)        
    elif jc==(n_1dim-1) and ic>0 and ic<(n_1dim-1): #a_central sur arête droite
        L=[(Matrice[ic-1 : ic+2,jc])
           ,(Matrice[ic-1 : ic+2,jc-1])
           ,(Matrice[ic-1 : ic+2,0])]
        L=np.reshape(L,9)        
    elif a_central==1:                              #a_central sur coin sup/gauche
        L=[n_1dim,a_central,2,n_1dim*2,n_1dim+1,n_1dim+2,n_1dim*n_1dim,n_1dim*(n_1dim-1)+1,n_1dim*(n_1dim-1)+2]
        L=np.array(L)        
    elif a_central==n_1dim:                                                    #a_central sur coin sup/droit
        L=[n_1dim-1,a_central,1,n_1dim*2-1,n_1dim*2,n_1dim+1,n_1dim*n_1dim-1,n_1dim*n_1dim,n_1dim*(n_1dim-1)+1]
        L=np.array(L)
    elif a_central==n_1dim*(n_1dim-1)+1:            #a_central sur coin inf/gauche
        L=[n_1dim*(n_1dim-1),Matrice[ic-1,jc],Matrice[ic-1,jc+1]
          ,n_1dim*n_1dim,a_central,Matrice[ic,jc+1]
          ,1,2,n_1dim]
        L=np.array(L)        
    elif a_central==(n_1dim*n_1dim):                #a_central sur coin inf/droit
        L=[a_central-n_1dim-1,a_central-n_1dim,Matrice[ic-1,0]
          ,a_central-1,a_central,a_central-n_1dim+1
          ,1,n_1dim-1,n_1dim]
        L=np.array(L)
    
    print('Voisinage de l\'agent central :',L)
    Ldiscu=np.random.choice(L,size,replace=False)
             
    return Ldiscu

Population=np.arange(1,N_1dim*N_1dim+1).reshape(N_1dim,N_1dim)
List_discu=selection_voisinage_bis(Population,N_1dim)
print('Population :\n',Population)
print('Liste des personnes qui vont discuter :',List_discu)

Agent central: 55
Voisinage de l'agent central : [44 55 66 43 54 65 34 45 56]
Population :
 [[  1   2   3   4   5   6   7   8   9  10  11]
 [ 12  13  14  15  16  17  18  19  20  21  22]
 [ 23  24  25  26  27  28  29  30  31  32  33]
 [ 34  35  36  37  38  39  40  41  42  43  44]
 [ 45  46  47  48  49  50  51  52  53  54  55]
 [ 56  57  58  59  60  61  62  63  64  65  66]
 [ 67  68  69  70  71  72  73  74  75  76  77]
 [ 78  79  80  81  82  83  84  85  86  87  88]
 [ 89  90  91  92  93  94  95  96  97  98  99]
 [100 101 102 103 104 105 106 107 108 109 110]
 [111 112 113 114 115 116 117 118 119 120 121]]
Liste des personnes qui vont discuter : [44 56 54 65 43]


In [32]:
#Modification du paramètre p de x en fonction de la force de persuasion de y et vice versa :
def force_persu(x, y, reseau):
        
    opx,deja_vux,fpx,px=reseau[str(x)]
    opy,deja_vuy,fpy,py=reseau[str(y)]
    if fpx>fpy:
        py=py+fpy*(px-py)
    elif fpy>fpx:
        px=px+fpx*(py-px)
    
    reseau[str(x)]=(opx,deja_vux,fpx,px)
    reseau[str(y)]=(opy,deja_vuy,fpy,py)
    
    return reseau


#Influence du voisinage :
    # si 2 personnes, force persu. Sinon, moyenne des p pondérée des fp; si abs(pi-moy)>seuil, pi=pi, sinon pi=moy
def influ_voisinage(list_discu,reseau,seuil):
    
    P=[]   #liste des convictions des membres de la discussion
    F=[]   #liste des forces de persuasion des membres de la discussion
    N=[]   #liste des noms des membres de la discussion
    i=0
    
    for k in list_discu:
        op_k,deja_k,fp_k,p_k=reseau[str(k)]
        N.append(int(k))
        P.append(p_k)
        F.append(fp_k)
            
    if len(N)==2:
        reseau=force_persu(N[0],N[1],reseau)
    else:
        for a in N:
            op_a,deja_a,fp_a,p_a=reseau[str(a)]
            other_p=P[0:a]+P[a+1:len(P)]
            other_fp=F[0:a]+F[a+1:len(F)]            
            moy=np.average(other_p,weights=other_fp)
            if abs(p_a-moy)<seuil:
                p_a=moy
            reseau[str(a)]=(op_a,deja_a,fp_a,p_a)
    
            
    return reseau

print(influ_voisinage(List_discu,Test_reseau,0.2))

{'1': (0, False, 0.7346610644792022, 0.6477871249978855), '2': (0, False, 0.45525028831027037, 0.26946477240128097), '3': (1, False, 0.912963782459134, 0.22072990873325526), '4': (1, False, 0.8862085155815076, 0.757202584897424), '5': (1, False, 0.9825138050903007, 0.9631282079132993), '6': (1, False, 0.028747756157672333, 0.900310729498335), '7': (0, True, 0.002107295007200416, 0.38632638413879294), '8': (0, False, 0.19336685203411685, 0.41955404803995444), '9': (1, True, 0.19662693644421148, 0.5973916975025212), '10': (1, True, 0.8032316100869032, 0.13855506366579617), '11': (0, False, 0.4008885277405718, 0.419208628445233), '12': (0, True, 0.23314933782228642, 0.27837861409227727), '13': (1, False, 0.689953603962745, 0.9893369133950484), '14': (0, False, 0.7325228084721132, 0.9629208077202279), '15': (0, True, 0.13208159666548458, 0.9773101286645968), '16': (0, True, 0.9943512396814158, 0.3566780732467155), '17': (1, False, 0.04211193981001771, 0.875672204483548), '18': (0, False, 0

## Modification du paramètre "deja_vu"

In [ ]:
#Modif deja_vu :
def deja_vu(reseau):
    for (nom,(op,deja_vu,fp,p)) in reseau.items():
        if p!=0.5:
            deja_vu=True
        reseau[nom]=(op,deja_vu,fp,p)
        
    return reseau

# Définition d'un pas de temps

Pendant un pas de temps, le paramètre p d'un ou plusieurs individus sera modifié

In [ ]:
def one_step(reseau,agent):
    

**Brouillons + tests + mémos numpy**

In [ ]:
"""numpy.random.choice(liste(ou entier) , taille de l'array , replace=False ne pas avoir 2 fois la meme valeur , proba)
   Parcours d'une matrice : for k in np.nditer(matrice)
   
N=15 #nombre de membres du réseau discussion physique
#description d'un agent : (nom,opinion,deja_vu,persuasion,chgt_opinion)
Reseau_discu={str(k):(0,False,fp,0.5) for k in range(N) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Noms_reseau_discu=[str(k) for k in range(N)]

size = int(np.random.choice(range(2,10)))
Discu = np.random.choice(Noms_reseau_discu,size,replace=False)
print('Membres de la discussion',Discu)

def matrice_discu(L,reseau):
    matrice=np.matrix([['00','00','00'],['00','00','00'],['00','00','00']])
    i=0
    j=0
    for (nom,(opinion,deja_vu,persu,p)) in reseau.items():
        if nom in L:
            matrice[i,j]=nom
            if j!=2:
                j=j+1
            elif j==2:
                j=0
                i=i+1
    return matrice
    
print('Matrice discussion :\n',matrice_discu(Discu.copy(),Reseau_discu))

Test_reseau=[(str(k),np.random.choice([0,1]),np.random.choice([False,True]),np.random.random_sample(),np.random.random_sample()) for k in range(N)]
print('Réseau aléatoire :',Test_reseau)

"""